Copiez le code que vous avez tapé dans le jupyter du tp2 dans un fichier .py. Il faudra modifier le prototype de la fonction cost_map_graphs qui sera:

cost_map_graphs(G1,G2,lab1,lab2,node_cost,node_del,node_ins,edge_cost,edge_del,edge_ins)


importez ce fichier grace à la commande import.

In [ ]:
from gklearn.utils.graphfiles import loadDataset
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from __future__ import print_function
import torch

import sys
sys.path.append('/home/luc/TRAVAIL/DeepGED/ESSAI_MUNKRES/gedlibpy')
import librariesImport
import gedlibpy
import tp2b

In [ ]:
Gs,y = loadDataset('./MAO/dataset.ds')



def label_to_color(label):
    if label == 'C':
        return 0.1
    elif label == 'O':
        return 0.8
    
def nodes_to_color_sequence(G):
    return [label_to_color(c[1]['label'][0]) for c in G.nodes(data=True)]


#for e in Gs[13].edges():
#    print(Gs[13][e[0]][e[1]]['bond_type'])
print('edge max label',max(max([[G[e[0]][e[1]]['bond_type'] for e in G.edges()] for G in Gs])))
G1 = Gs[7]
G2 = Gs[15]
print(y[1],y[9])
plt.figure(0)
nx.draw_networkx(G1,with_labels=True,node_color = nodes_to_color_sequence(G1),cmap='autumn')

#plt.figure(1)
#nx.draw_networkx(G2,with_labels=True,node_color = nodes_to_color_sequence(G2),cmap='autumn')

dict,nb_edge_labels=tp2b.build_node_dictionnary(Gs)


node_cost=np.full((len(dict),len(dict)),1.0)
node_del=2.0
node_ins=2.0
node_cost[np.diag_indices(len(dict))]=0.0
                   
edge_cost=np.full((nb_edge_labels+1,nb_edge_labels+1),.5)
edge_del=1.0
edge_ins=1.0
edge_cost[np.diag_indices(nb_edge_labels+1)]=0.0



Le code ci dessous (la fonction construct_cost_matrix) renvoie la matrice $\Delta$ et le vecteur $c$, vectorisation de la matrice $C$  mentionnés page 125, 126 du cours. Completez le code du Franck Wolfe donné à la page 143 du cours. Le calcul de $b^*$ se fait à l'aide d'un algorithme LSAPE appliqué sur la matrice $Cp$. Il s'agit à partir de cette matrice de retourner un vecteur de taille $(n+1)(m+1)\times 1$ codant la matrice $X$ d'$\epsilon$ assignement (p. 122) qui minimise :
$$
\sum_{i=1}^{n+1}\sum_{j=1}^{m+1}Cp[i,j]X[i,j]
$$

In [ ]:
def matrix_edgeInsDel(A1,A2):
    Abin1=(A1!=torch.zeros((A1.shape[0],A1.shape[1])))
    Abin2=(A2!=torch.zeros((A2.shape[0],A2.shape[1])))
    C1=torch.einsum('ij,kl->ijkl',torch.logical_not(Abin1),Abin2)
    C2=torch.einsum('ij,kl->ijkl',Abin1,torch.logical_not(Abin2))
    C12=torch.logical_or(C1,C2).int()
    
    return torch.cat(torch.unbind(torch.cat(torch.unbind(C12,1),1),0),1)
 
def matrix_edgeSubst(A1,A2,lab1,lab2):
        Abin1=(A1==lab1*torch.ones((A1.shape[0],A1.shape[1]))).int()
        Abin2=(A2==lab2*torch.ones((A2.shape[0],A2.shape[1]))).int()
        C=torch.einsum('ij,kl->ijkl',Abin1,Abin2)
        
        return torch.cat(torch.unbind(torch.cat(torch.unbind(C,1),1),0),1).float()

def construct_cost_matrix(M1,M2,lab1,lab2,node_costs,edge_costs,nodeInsDel,edgeInsDel,nb_edge_labels):
    n = M1.shape[0]
    m = M2.shape[0]
    A1=torch.zeros((n+1,n+1),dtype=torch.int)
    A1[0:n,0:n]=torch.from_numpy(M1)
    A2=torch.zeros((m+1,m+1),dtype=torch.int)
    A2[0:m,0:m]=torch.from_numpy(M2)
    with torch.no_grad():            
        A=matrix_edgeInsDel(A1,A2)
        
         # costs: 0 node subs, 1 nodeIns/Del, 2 : edgeSubs, 3 edgeIns/Del
        
        #C=cost[3]*torch.cat([torch.cat([C12[l][k] for k in range(n+1)],1) for l in range(n+1)])
        #Pas bien sur mais cela semble fonctionner.
    C=edgeInsDel*A
    if nb_edge_labels>1:
        for k in range(nb_edge_labels):
            for l in range(nb_edge_labels):
                if k != l:
                    C.add_(matrix_edgeSubst(A1,A2,k+1,l+1).multiply_(edge_costs[k][l]))

                        
        #C=cost[3]*torch.tensor(np.array([ [  k!=l and A1[k//(m+1),l//(m+1)]^A2[k%(m+1),l%(m+1)] for k in range((n+1)*(m+1))] for l in range((n+1)*(m+1))]),device=self.device)        

    l1=torch.from_numpy(lab1)
    l2=torch.from_numpy(lab2)
    D=torch.zeros((n+1)*(m+1))
    D[n*(m+1):]=nodeInsDel
    D[n*(m+1)+m]=0
    D[[i*(m+1)+m for i in range(n)]]=nodeInsDel
    for k in range(n*(m+1)):
        if k%(m+1) != m:
            D[k]=node_costs[l1[k//(m+1)]][l2[k%(m+1)]]#self.get_node_costs(l1[k//(m+1)],l2[k%(m+1)])
       
        #D[[k for k in range(n*(m+1)) if k%(m+1) != m]]=torch.tensor([node_costs[l1[k//(m+1)],l2[k%(m+1)]] for k in range(n*(m+1)) if k%(m+1) != m],device=self.device )
    #with torch.no_grad():
    #    mask = torch.diag(torch.ones_like(D))
   
    #C=mask*torch.diag(D) + (1. - mask)*C
        
        #C[range(len(C)),range(len(C))]=D
      
    return C.numpy(),D.numpy().reshape((1,(n+1)*(m+1)))



In [ ]:
def franck_wolfe(x0,D,c,kmax,n,m):
    k=0
    converged=False
    x=x0        
    print('D shape',D.shape,'c shape:',c.shape,'x0 shape:',x0.shape)
    while (not converged) and (k<=kmax):
        Cp=(x.T@D+c).reshape(n+1,m+1)
       #to be completed.
        alpha=x.T@D@(b-x)+c@(b-x)       

        if alpha >0: # security check if b is not a local minima (does not occur with real hungarian)
            print('alpha positif(',k,')',alpha.item())
            return x

        beta=.5*(b-x).T@D@(b-x)
        if beta <=0:
            t=1.0
        else:
            t=min(-alpha/(2*beta),1.0)
        x=x+t*(b-x)
        k=k+1
        converged= (-alpha < 1e-5)
#        print('cost(',k,')=',.5*x.T@D@x+c.T@x,'-alpha=',-alpha)
#        print('converge=',converged)
    
        

    return x,.5*(x.T@D@x+c@x)

Completez la fonction ged_fw qui :
1. calcule une solution initiale $x0$. On utilisera pour cela la fonction d'upper bound de la ged calculée au tp2. 
2. initialise les matrices d'adjacences A1, A2 de G1 et G2 ansi que les ensembles de numéro de labels des noeuds (lab1 et lab2)
3. Appelle construct_cost_matrix avec A1,A2,lab1,lab2 et les fonctions de couts sur les noeuds et les arêtes
4. utilise le retour de construct_cost_matrix et la solution initiale x0 pour appeler franck_wolfe. On fixera le nombre d'itérations maximum à 15.
5. Renvoie la solution sous la forme d'une matrice $(n+1)\times(m+1)$ et de son coût associé (n,m : tailles de G1 et G2).

In [ ]:
def ged_fw(G1,G2):
    #to be completed.

    return x.reshape(X.shape[0],X.shape[1]),ged

G1 = Gs[8]
G2 = Gs[12]

x,ged=ged_fw(G1,G2)
print(x,'ged=',ged)
#print(matrix_edgeInsDel(torch.from_numpy(A1),torch.from_numpy(A2)),matrix_edgeSubst(torch.from_numpy(A1),torch.from_numpy(A2),1,2))

L'idée à présent est d'utiliser nos trois métriques (lower bound, upper bound, franck_wolfe) pour classifier les molécules de MAO. On va pour cela définir arbitrairement les n premières molécules de Gs comme le test set le reste correspondant au train set. Le cardinal n doit être défini comme un pourcentage de la taille du tableau (10%, 20%,...). Si N est la taille de la base :
1. Définissez une matrice $n\times (N-n), D$ codant la distance (lower ou upper bound) entre les éléments du test set et ceux du train set. 
2. Calculez les $k$ plus proches voisins dans le train set de chaque molécule du test set.
3. Prédisez la classe de chaque molécule en fonction de la classe majoritaire de ses k plus proches voisins. 
4. Calculez la précision et l'accuracy en faisant varier k, la taille du test set/train set et la métrique (lower/upper bound/Franck Wolfe).

On pourra pour cela utiliser les fonctions argsort et count_nonzero. 

In [ ]:
nb=len(Gs)
test_set=int(nb*.2)

    
    
#closest0=np.count(y[closest[:,0:3]]==0)
#closest1=np.count(y[closest[:,0:3]]==1)